# Connect to Snowflake

In [31]:
# access data from snowflake
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

connection_parameters = {
    "account": "",
    "user": "", 
    "host": "",
    "password": "",
    "role": "ACCOUNTADMIN",
    "warehouse": "SMALL_WH",
    "database":"MFR",
    "schema":"PUBLIC"
    }
session = Session.builder.configs(connection_parameters).create()

maintenance_df = session.table('MFR.PUBLIC.maintenance')
humidity_df = session.table('MFR.PUBLIC.Humidity')
hum_udi_df = session.table('MFR.PUBLIC.HUMIDITY_UDI')

# Look at Each of the dataframes

In [32]:
maintenance_df.to_pandas().head()

,UDI,PRODUCT_ID,TYPE,AIR_TEMPERATURE_K,PROCESS_TEMPERATURE,ROTATIONAL_SPEED_RPM,TORQUE_NM,TOOL_WEAR_MIN,MACHINE_FAILURE,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298,309,1551,43,0,0,0,0,0,0,0
1,2,L47181,L,298,309,1408,46,3,0,0,0,0,0,0
2,3,L47182,L,298,309,1498,49,5,0,0,0,0,0,0
3,4,L47183,L,298,309,1433,40,7,0,0,0,0,0,0
4,5,L47184,L,298,309,1408,40,9,0,0,0,0,0,0


In [3]:
humidity_df.to_pandas().head()

,CITY_NAME,HUMIDITY_RELATIVE_AVG
0,Riyadh,71
1,Singapore,78
2,Los Angeles,63
3,Sydney,78
4,Kabul,57


In [4]:
hum_udi_df.to_pandas().head()

,UDI,CITY
0,1,Santiago
1,2,Dakar
2,3,Dhaka
3,4,Los Angeles
4,5,Baghdad


In [5]:
# join together the dataframes and prepare training dataset
maintenance_city = maintenance_df.join(hum_udi_df, ["UDI"])
maintenance_hum = maintenance_city.join(humidity_df, (maintenance_city.col("CITY") == humidity_df.col("CITY_NAME"))).select(col("TYPE"), 
col("AIR_TEMPERATURE_K"), col("PROCESS_TEMPERATURE"), col("ROTATIONAL_SPEED_RPM"), col("TORQUE_NM"), col("TOOL_WEAR_MIN"), col("HUMIDITY_RELATIVE_AVG"), col("MACHINE_FAILURE"))

In [6]:
# write training set to snowflake and materialize the data frame into a pandas data frame
maintenance_hum.write.mode("overwrite").save_as_table("MFR.PUBLIC.MAINTENANCE_HUM")
maintenance_hum_df = session.table('MFR.PUBLIC.MAINTENANCE_HUM').to_pandas()

In [7]:
# convert categorical to dummies
from pandas import get_dummies
# maintenance_hum_df = pd.concat([get_dummies(maintenance_hum_df[["TYPE"]]),maintenance_hum_df.drop(columns=["TYPE"])], axis = 1)
maintenance_hum_df = maintenance_hum_df.drop(columns=["TYPE"])

# Use MLFlow to track jobs and models

In [ ]:
import mlflow
mlflow.set_experiment(experiment_name="predictive-maintenance2")

# Split data and train model

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    maintenance_hum_df.drop("MACHINE_FAILURE", axis=1), maintenance_hum_df["MACHINE_FAILURE"], test_size=0.3
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
mlflow.autolog()
model = RandomForestClassifier()


In [11]:
run = mlflow.start_run()

In [ ]:
model.fit(X_train, y_train)

In [13]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))

In [16]:
mlflow.end_run()

In [17]:
run = mlflow.get_run(run.info.run_id)

# Look at model metrics

In [ ]:
pd.DataFrame(data=[run.data.params], index=["Value"]).T

In [ ]:
pd.DataFrame(data=[run.data.metrics], index=["Value"]).T

In [ ]:
# evaluate model on test
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay
import matplotlib.pyplot as plt
y_pred = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                   estimator_name='example estimator')
display.plot()
plt.show()

In [ ]:
# auc score
roc_auc_score(y_test, y_pred)

In [ ]:
from sklearn.inspection import permutation_importance
feature_names = ['AIR_TEMPERATURE_K',
       'PROCESS_TEMPERATURE', 'ROTATIONAL_SPEED_RPM', 'TORQUE_NM',
       'TOOL_WEAR_MIN', 'HUMIDITY_RELATIVE_AVG']
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=feature_names)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

# Register training dataset with mlflow runid

In [ ]:
# register dataset
from azureml.core import Workspace, Dataset

# Connect to the Workspace
ws = Workspace.from_config()

# The default datastore is a blob storage container where datasets are stored
datastore = ws.get_default_datastore()

# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=maintenance_hum_df, 
        name=run.info.run_id+'maintenance_hum_df', 
        description='maintenance df for model training',
        target=datastore
    )

In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run.info.run_id)

# Register Model

In [ ]:
import os
mlflow.sklearn.save_model(model, "./maint")
model_local_path = os.path.abspath("./maint")
mlflow.register_model(f"file://{model_local_path}", "maint-model")

# Deploy Model to snowflake

In [ ]:
# create stage
session.sql("CREATE OR REPLACE STAGE MFR.PUBLIC.maint_stage").collect()

In [ ]:
# push model to stage in snowflake
# Model.get_model_path(model_name = 'mfr_model', version = 1, _workspace= ws)
session.file.put('maint/model.pkl', "MFR.PUBLIC.maint_stage", auto_compress=False, overwrite=True)

In [29]:
sql_cmd = '''
create or replace function predict_failure(a int, b int,c int,d int, 
                                     e int, f int)
returns float
language python
runtime_version = 3.8
packages = ('scikit-learn', 'pandas')
handler = 'udf'
imports = ('@maint_stage/model.pkl')
as $$
import pandas
import sklearn
import sys
import pickle
from _snowflake import vectorized

IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
pipe_file = import_dir + 'model.pkl'
with open(pipe_file,'rb') as f:
         pred_model = pickle.load(f)
         
@vectorized(input=pandas.DataFrame)
def udf(df):
    return pred_model.predict_proba(df)[:,1]
$$;
'''

In [33]:
session.sql(sql_cmd).collect()

[Row(status='Function PREDICT_FAILURE successfully created.')]